# Методы сбора и обработки данных из сети Интернет
## Урок 1. Основы клиент-серверного взаимодействия. Работа с API

1. Посмотреть документацию к API GitHub, разобраться как вывести список репозиториев для конкретного пользователя, сохранить JSON-вывод в файле *.json.



Ссылка на документацию: [List repositories for a user](https://docs.github.com/en/rest/repos/repos#list-repositories-for-a-user)

In [104]:
import requests

username = 'Lauarvik' #это я 
url = f'https://api.github.com/users/{username}/repos'
response = requests.get(url)
if response.status_code == 200:
    j_data = response.json()
    print(f'Список репозиториев для пользователя {username}')
    for i in j_data:
        print(i['name'],':', i['description'])
    with open(f'{username}_repos.json', 'w', encoding='utf-8') as f:
        f.write(str(j_data))
        

Список репозиториев для пользователя Lauarvik
python : Основы языка Python
python_data_science : Библиотеки Python для Data Science
python_scraping : Методы сбора и обработки данных из сети Интернет


2. Работа будет состоять с недокументированным API. Нужно ввести релевантный запрос на сайте https://www.delivery-club.ru/search

(а) из предложенных точек с помощью API найти долю (в %) с бесплатной и платной доставкой. Для каждой категории рассчитать среднюю минимальную стоимость заказа.

(б) для каждой из категорий из пункта (а) рассчитать долю (в %) магазинов и ресторанов

In [105]:
from time import time
import pandas as pd
import requests

query = 'суши'

params = {
    'latitude': 55.7577374,
    'longitude': 37.6164793,
    'query': query,
    'cacheBreaker': int(time()),

}
url = 'https://api.delivery-club.ru/api1.2/vendors/search'
response = requests.get(url, params=params)
if response.status_code == 200:
    j_data = response.json()
    df = pd.json_normalize(j_data['vendors'])

In [106]:
paid_delivery = df[df['delivery.price.value'] > 0]
free_delivery = df[df['delivery.price.value'] == 0]
print('Заведений с платной доставкой: {:.0%}'.format(len(paid_delivery) / (len(paid_delivery) + len(free_delivery))))
print('Заведений с бесплатной доставкой: {:.0%}'.format(len(free_delivery) / (len(paid_delivery) + len(free_delivery))))

Заведений с платной доставкой: 66%
Заведений с бесплатной доставкой: 34%


In [107]:
#средняя минимальная стоимость заказа для категории "платная доставка"
paid_delivery['delivery.minOrderPrice.value'].mean()

84.17391304347827

In [108]:
#средняя минимальная стоимость заказа для категории "бесплатная доставка"
free_delivery['delivery.minOrderPrice.value'].mean()

1262.0

In [109]:
paid_delivery_shops = paid_delivery[paid_delivery['categoryId'] == 3]
paid_delivery_restaurants = paid_delivery[paid_delivery['categoryId'] == 1]
free_delivery_shops = free_delivery[free_delivery['categoryId'] == 3]
free_delivery_restaurants = free_delivery[free_delivery['categoryId'] == 1]
print('Магазинов с платной доставкой: {:.0%}'.format(len(paid_delivery_shops) / (len(paid_delivery_shops) + len(paid_delivery_restaurants))))
print('Ресторанов с платной доставкой: {:.0%}'.format(len(paid_delivery_restaurants) / (len(paid_delivery_shops) + len(paid_delivery_restaurants))))
print('Магазинов с бесплатной доставкой: {:.0%}'.format(len(free_delivery_shops) / (len(free_delivery_shops) + len(free_delivery_restaurants))))
print('Ресторанов с бесплатной доставкой: {:.0%}'.format(len(free_delivery_restaurants) / (len(free_delivery_shops) + len(free_delivery_restaurants))))


Магазинов с платной доставкой: 2%
Ресторанов с платной доставкой: 98%
Магазинов с бесплатной доставкой: 0%
Ресторанов с бесплатной доставкой: 100%
